<a href="https://colab.research.google.com/github/Ericlinyuting/Web_Crawler_591/blob/main/newhouse591_without_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (136 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0

In [13]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import datetime
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
import io
import json
import random
warnings.filterwarnings("ignore")

In [14]:
headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }

request_url='http://newhouse.591.com.tw/home/housing/info?hid=119282'
res=requests.get(request_url, headers = headers)
bs=BeautifulSoup(res.text,'html.parser')


In [ ]:
# input 建案網址 return 建案建案詳情欄位
# 產出為建案名與12個建案資訊
def getData(url):
    request_url='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()
    headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
    res=requests.get(request_url, headers = headers)
    #bs=BeautifulSoup(res.text,'html.parser')

    request_url_detail='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()+"/detail"
    res_detail=requests.get(request_url_detail, headers = headers)

    if res.status_code == 200 & res_detail.status_code==200:
        bs=BeautifulSoup(res.text,'html.parser')
        bs_detail=BeautifulSoup(res_detail.text,'html.parser')
        #先宣告變數為NULL 若無撈到資料則寫入NULL
        title='NULL'
        tag='NULL'
        unit_price='NULL'
        unit='NULL'
        material='NULL'
        htype='NULL'
        htype2='NULL'
        htype3='NULL'
        htype4='NULL'
        htype5='NULL'
        htype6='NULL'
        htype7='NULL'
        htype8='NULL'
        htype9='NULL'
        htype10='NULL'
        htype11='NULL'

        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
        title=bs.find('h1').text
        tag = ', '.join([span.text.strip() for span in bs.find("p", {'class':"build-tag"}).find_all("span")]) if bs.find("p", {'class':"build-tag"}) else "未找到相應標籤" #建案標籤
        unit_price=bs.find("span", {'class':"price"}).text #單價
        unit = bs.find("span", {'class':"unit"}).text #單位
        material = bs.find("h4",text="建材說明").findNext("p").text #建材說明
        htype=bs_detail.find("span", text="建案類別").findNext("p").string.strip() #建案類別
        htype2=bs_detail.find("span", text="建案型態").findNext("p").string.strip().replace(' ', '').replace('\n', '、') #建案型態
        htype3=bs_detail.find("span", text="公開銷售").findNext("p").string.strip() #公開銷售
        htype4=bs_detail.find("p", {'class':"address"}).findNext("span").text #基地地址
        htype5=bs_detail.find("span", text="交屋屋況").findNext("p").string.strip() #交屋屋況
        htype6=bs_detail.find("span",text="格局規劃").findNext("p").text.strip().replace(' ', '') #格局規劃
        htype7=bs_detail.find("span", text="投資建設").findNext("p").string.strip() #投資建設
        htype8=bs_detail.find("span", text="營造公司").findNext("p").string.strip() #營造公司
        htype9=bs_detail.find("span", text="棟戶規劃").findNext("p").string.strip() #棟戶規劃
        htype10=bs_detail.find("span", text="樓層規劃").findNext("p").string.strip() #樓層規劃
        hhtype11=bs_detail.find("span", text="用途規劃").findNext("p").string.strip() #用途規劃

        return title,tag,unit_price,unit,material,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11
    else:
        print('link expired:', url)
        return 404, 404, 404, 404, 404, 404, 404

In [ ]:
#def get_dynamic_data(url):
#    request_url='https://newhouse.591.com.tw/home/housing/dynamic?hid='+str(url).strip()
#    headers={
#    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
#    }
#    res=requests.get(request_url, headers = headers)
#    #bs=BeautifulSoup(res.text,'html.parser')
#
#    if res.status_code == 200:
#        bs=BeautifulSoup(res.text,'html.parser')
#        #先宣告變數為NULL 若無撈到資料則寫入NULL
#        news_date = 'NULL'
#        news = 'NULL'
#        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
#        news_date = bs.find("div", {'class':"dynamic-date"}).findNext("span").contents[0].string.strip() + "-" +\
#                    bs.find("div", {'class':"dynamic-date"}).findNext("strong").contents[0].string.strip()
#
#        news = bs.find("div", {'class':"dynamic-infolist"}).string
#
#        return news_date, news
#    else:
#        print('link expired:', url)
#        return 404, 404, 404, 404, 404, 404, 404

In [43]:
request_url = "https://newhouse.591.com.tw/home/housing/search?rid=1&sid=0&page=1"
headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
response = requests.get(request_url, headers = headers)
response = response.json()
items = response["data"]["items"]
house_url_list=[] #存放網址list
for key in items:
  id = key["hid"] # 每個物件的 id
  house_url_list.append(id)

for url in house_url_list:
  request_url='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()+"/detail"
  headers={
  'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
  }
  time.sleep(random.randint(5,10))
  res=requests.get(request_url, headers = headers)

print (request_url)
print (res)

https://newhouse.591.com.tw/home/housing/info?hid=130398/detail
<Response [200]>


In [98]:
request_url='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()
headers={
  'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
  }
res=requests.get(request_url, headers = headers)
bs=BeautifulSoup(res.text,'html.parser')
# 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
title=bs.find('h1').text
tag = ', '.join([span.text.strip() for span in bs.find("p", {'class':"build-tag"}).find_all("span")]) if bs.find("p", {'class':"build-tag"}) else "未找到相應標籤" #建案標籤
unit_price=bs.find("span", {'class':"price"}).text #單價
unit = bs.find("span", {'class':"unit"}).text #單位
room = bs.find("span",text="格局").findNext("span",{"class":"value"}).text #格局
material = bs.find("h4",text="建材說明").findNext("p").text #建材說明
print (res)
print (material)

<Response [200]>
Doris廚具、innoci衛浴、住友制震壁


In [97]:
request_url_detail='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()+"/detail"
headers={
  'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
  }
res_detail=requests.get(request_url_detail, headers = headers)
bs_detail=BeautifulSoup(res_detail.text,'html.parser')

htype=bs_detail.find("span", text="建案類別").findNext("p").string.strip()
htype2=bs_detail.find("span", text="建案型態").findNext("p").string.strip().replace(' ', '').replace('\n', '、')
htype3=bs_detail.find("span", text="公開銷售").findNext("p").string.strip()
htype4=bs_detail.find("p", {'class':"address"}).findNext("span").text
htype5=bs_detail.find("span", text="交屋屋況").findNext("p").string.strip()
htype6=bs_detail.find("span",text="格局規劃").findNext("p").text.strip().replace(' ', '')
htype7=bs_detail.find("span", text="投資建設").findNext("p").string.strip()
htype8=bs_detail.find("span", text="營造公司").findNext("p").string.strip()
htype9=bs_detail.find("span", text="棟戶規劃").findNext("p").string.strip()
htype10=bs_detail.find("span", text="樓層規劃").findNext("p").string.strip()
htype11=bs_detail.find("span", text="用途規劃").findNext("p").string.strip()
print(htype11)

住商用


In [99]:
def main(outputfile, rid, sid, totalpages):

   with io.open(outputfile, "w", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([ "建案名稱", "建案標籤","單價","單位","建材說明" ,"建案類別", "建物形態", "公開銷售", "基地地址", "交屋屋況"\
                        ,"格局規劃", "投資建設", "營造公司", "棟戶規劃", "樓層規劃", "用途規劃", "網址"]) \
                        #,"動態資訊日期日期", "動態資訊"])
        totalpages = totalpages
        print('Total pages: ', totalpages)

        for i in range(1, totalpages+1):
            request_url = "https://newhouse.591.com.tw/home/housing/search?rid="+str(rid)+"&sid="+str(sid)+"&page="+str(i)
            response = requests.get(request_url, headers = headers)
            response = response.json()
            items = response["data"]["items"]

            house_url_list=[] #存放網址list
            for key in items:
                id = key["hid"] # 每個物件的 id
                house_url_list.append(id)
            #if totalpages%14 == 0:
            #  time.sleep(180)
            #else:
            time.sleep(random.randint(5,10))

            # ------------- write into csv ------------- #
            for url in house_url_list:
                title,cost,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11 = getData(url)
                #news_date, news = get_dynamic_data(url)
                writer.writerow([title,cost,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11,\
                                 str('https://newhouse.591.com.tw/home/housing/info?hid='+str(url))])
                                 #, news_date, news])
            # ------------------------------------------ #
            print(i/totalpages*100, '%') # print out 完成 %數

SyntaxError: ignored

In [ ]:
if __name__ == '__main__':
    # -------- configurable parameter -------- #
    # 以台北市不限區舉例(預設網址可能沒寫rid&sid, 點選縣市或區往只會顯示如下)
    # link:https://newhouse.591.com.tw/housing-list.html?rid=1&sid=0

    output_file_name = '/content/drive/MyDrive/output.csv' #設定存放位置與檔名
    rid = 1           # 設定縣市 (台北市 rid = 1)
    sid = 0           # 設定地區 (不限區 sid = 0)
    totalpages = 35    # 設定抓取頁數 # 14頁大概
    # ---------------------------------------- #

    main(output_file_name, rid, sid, totalpages)                                          #匯出csv檔
    read_file = pd.read_csv (output_file_name)
    read_file.to_excel ('/content/drive/MyDrive/output.xlsx', index = None, header=True) #匯出excel檔
    print('\nfinish!')

Total pages:  35
42.857142857142854 %
45.714285714285715 %
48.57142857142857 %
51.42857142857142 %
54.285714285714285 %
57.14285714285714 %
60.0 %
62.857142857142854 %
65.71428571428571 %
68.57142857142857 %
71.42857142857143 %
74.28571428571429 %
77.14285714285715 %
80.0 %
82.85714285714286 %
85.71428571428571 %
88.57142857142857 %
91.42857142857143 %
94.28571428571428 %
97.14285714285714 %
100.0 %

finish!
